In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM, Flatten
from keras.layers.embeddings import Embedding
from collections import Counter

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
pdb_data_no_dups = pd.read_csv('/content/drive/MyDrive/599/pdb_data_no_dups.csv')
pdb_data_seq = pd.read_csv('/content/drive/MyDrive/599/pdb_data_seq.csv')
df = pdb_data_no_dups.merge(pdb_data_seq, how='inner', on='structureId')

# Drop rows with missing labels
df = df[[type(c) == type('') for c in df.classification.values]]
df = df[[type(c) == type('') for c in df.sequence.values]]
# select proteins
df = df[df.macromoleculeType_x == 'Protein']
#df.reset_index()

# count numbers of instances per class
cnt = Counter(df.classification)
top_classes = 10
# sort classes
sorted_classes = cnt.most_common()[:top_classes]
classes = [c[0] for c in sorted_classes]
counts = [c[1] for c in sorted_classes]
df = df[[c in classes for c in df.classification]]

seqs = df.sequence.values
# Transform labels to one-hot
lb = LabelBinarizer()
Y = lb.fit_transform(df.classification)

max_length = 256

#create and fit tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(seqs)
#represent input data as word rank number sequences
X = tokenizer.texts_to_sequences(seqs)
X = sequence.pad_sequences(X, maxlen=max_length)



In [3]:
embedding_dim = 8

#  model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(Dense(top_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 256, 8)            208       
_________________________________________________________________
lstm (LSTM)                  (None, 256, 128)          70144     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 203,226
Trainable params: 203,226
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#  train
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=256)

Epoch 1/50
589/589 [==============================] - 67s 57ms/step - loss: 2.0277 - accuracy: 0.2456 - val_loss: 2.0575 - val_accuracy: 0.2467
Epoch 2/50
589/589 [==============================] - 34s 57ms/step - loss: 2.0552 - accuracy: 0.2489 - val_loss: 2.0529 - val_accuracy: 0.2492
Epoch 3/50
589/589 [==============================] - 34s 58ms/step - loss: 1.9723 - accuracy: 0.2649 - val_loss: 1.9258 - val_accuracy: 0.2831
Epoch 4/50
589/589 [==============================] - 34s 57ms/step - loss: 1.8258 - accuracy: 0.3140 - val_loss: 2.2482 - val_accuracy: 0.2362
Epoch 5/50
589/589 [==============================] - 34s 58ms/step - loss: 1.7910 - accuracy: 0.3248 - val_loss: 1.6373 - val_accuracy: 0.3644
Epoch 6/50
589/589 [==============================] - 34s 58ms/step - loss: 1.6032 - accuracy: 0.3870 - val_loss: 1.5429 - val_accuracy: 0.4176
Epoch 7/50
589/589 [==============================] - 34s 57ms/step - loss: 1.5931 - accuracy: 0.4034 - val_loss: 1.9238 - val_accuracy: